In [1]:
import re
import pandas as pd
import numpy as np
from scipy.stats import spearmanr, pearsonr
import matplotlib.pyplot as plt
plt.style.use('ggplot')

In [2]:
df_str_rel = pd.read_csv('C:\\Users\\ADMIN\\Downloads\\amh_train.csv')
df_str_rel.head()

,PairID,Text,Score
0,Pair_ID_amh_train_1,መግለጫውን የተከታተለው የአዲስ አበባው ዘጋቢያችን ሰሎሞን ሙጬ ዝርዝር ዘ...,0.88
1,Pair_ID_amh_train_2,የኛ ዴሞክራሲ የእንግሊዝ ስርዓት ነው ፤ ለጊዜው ሊያገለግል ይችላል ።\t...,0.25
2,Pair_ID_amh_train_3,በዛሬው ጊዜ ከፍቅራዊ ደግነቱ መጠቀም የምንችለውስ እንዴት ነው ?\tጥንቃ...,0.36
3,Pair_ID_amh_train_4,ግን ምን አይነት ቁልፍ ?\tምን አይነት ፍቅር ነዉ ?,0.27
4,Pair_ID_amh_train_5,ምን አይነት ተንኮለኛ ነው ?\tአጭር ሰው ግን የምር ተንኮለኛ ነው ?,0.56


In [3]:
df_str_rel['Split_Text'] = df_str_rel['Text'].apply(lambda x: x.split("\n"))
df_str_rel.head()

,PairID,Text,Score,Split_Text
0,Pair_ID_amh_train_1,መግለጫውን የተከታተለው የአዲስ አበባው ዘጋቢያችን ሰሎሞን ሙጬ ዝርዝር ዘ...,0.88,[መግለጫውን የተከታተለው የአዲስ አበባው ዘጋቢያችን ሰሎሞን ሙጬ ዝርዝር ...
1,Pair_ID_amh_train_2,የኛ ዴሞክራሲ የእንግሊዝ ስርዓት ነው ፤ ለጊዜው ሊያገለግል ይችላል ።\t...,0.25,[የኛ ዴሞክራሲ የእንግሊዝ ስርዓት ነው ፤ ለጊዜው ሊያገለግል ይችላል ።\...
2,Pair_ID_amh_train_3,በዛሬው ጊዜ ከፍቅራዊ ደግነቱ መጠቀም የምንችለውስ እንዴት ነው ?\tጥንቃ...,0.36,[በዛሬው ጊዜ ከፍቅራዊ ደግነቱ መጠቀም የምንችለውስ እንዴት ነው ?\tጥን...
3,Pair_ID_amh_train_4,ግን ምን አይነት ቁልፍ ?\tምን አይነት ፍቅር ነዉ ?,0.27,[ግን ምን አይነት ቁልፍ ?\tምን አይነት ፍቅር ነዉ ?]
4,Pair_ID_amh_train_5,ምን አይነት ተንኮለኛ ነው ?\tአጭር ሰው ግን የምር ተንኮለኛ ነው ?,0.56,[ምን አይነት ተንኮለኛ ነው ?\tአጭር ሰው ግን የምር ተንኮለኛ ነው ?]


In [8]:
from transformers import AutoModel, AutoTokenizer
import torch
from sklearn.metrics.pairwise import cosine_similarity

# Load the AfriBERTa model and tokenizer
model_name = "castorini/afriberta_small"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

# Function to encode text and get embeddings
def encode_text(text):
    input_ids = tokenizer(text, return_tensors="pt", padding=True, truncation=True)["input_ids"]
    with torch.no_grad():
        embeddings = model(input_ids).last_hidden_state.mean(dim=1)  # Average pooling
    return embeddings

# Calculate cosine similarity between two sentence embeddings
def calculate_similarity(sentence1, sentence2):
    embeddings1 = encode_text(sentence1)
    embeddings2 = encode_text(sentence2)
    similarity = cosine_similarity(embeddings1, embeddings2)
    return similarity[0][0]


Some weights of XLMRobertaModel were not initialized from the model checkpoint at castorini/afriberta_small and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:
true_scores = df_str_rel['Score'].values
pred_scores = []

for index,row in df_str_rel.iterrows():
  s1,s2 = row["Text"].split("\n")

  # Overlap score
  pred_scores.append(calculate_similarity(s1,s2))

ValueError: not enough values to unpack (expected 2, got 1)

In [ ]:
print("Spearman Correlation:", round(spearmanr(true_scores,pred_scores)[0],2))